In [1]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset

from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [2]:
class Embedding:
    #CLS is a special classification token and the last hidden state of BERT Embedding
    def cls_pooling(self, model_output):
        return model_output.last_hidden_state[:, 0]

    #BERT tokenizer of input text
    def get_embeddings(self, text_list):
        encoded_input = tokenizer(
            text_list, padding=True, truncation=True, return_tensors="pt"
        )
        encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
        model_output = model(**encoded_input)
        return self.cls_pooling(model_output).cpu().detach().numpy()

In [9]:


class Faiss:
    def __init__(self):
        pass
    
    #convert dataset into embeddings dataset to run FAISS
    def makeEmbeddings(self,dataset,df=None):
        embeddings = []
        for data in dataset:
            embeddings.append(Embedding().get_embeddings(data)[0])
        embeddings_dataset = pd.DataFrame(
            {
                "embeddings":embeddings,
                "values":dataset
            })
        if (df is not None):
            embeddings_dataset = pd.concat([df, embeddings_dataset], axis=1)
        embeddings_dataset = Dataset.from_pandas(embeddings_dataset)
        return embeddings_dataset
    
    # run faiss model on dataset
    def faiss(self,embeddings_dataset):
        embeddings_dataset.add_faiss_index(column="embeddings")
    
    #get query embedding
    def getQueryEmbedding(self, query):
        return Embedding().get_embeddings([query])
        
    #predict 4 nearest neighbors
    def predict(self,query,embeddings_dataset,k=4):
        query_embedding = self.getQueryEmbedding(query)
        scores, samples = embeddings_dataset.get_nearest_examples("embeddings", query_embedding, k=5)
        samples = pd.DataFrame(samples)
        samples["scores"] = scores
        return samples
        

In [10]:
values = ["julia","vedha","isabelle"]
f = Faiss()
embeddings_dataset = f.makeEmbeddings(values)
f.faiss(embeddings_dataset)
f.predict("vedha",embeddings_dataset)

  0%|          | 0/1 [00:00<?, ?it/s]

,embeddings,values,scores
0,"[-0.15023988485336304, -0.4446641504764557, -0...",vedha,0.000000
1,"[0.04670529440045357, -0.22696265578269958, -0...",julia,59.554565
2,"[0.15655475854873657, -0.3943760395050049, -0....",isabelle,64.213142


In [5]:
df = pd.read_csv("SEC-CompanyTicker.csv",index_col=0)
df = df.head(100)
dataset = df.companyName.tolist()
f = Faiss()
embeddings_dataset = f.makeEmbeddings(dataset)
f.faiss(embeddings_dataset)
f.predict("goldman",embeddings_dataset)

  0%|          | 0/1 [00:00<?, ?it/s]

,embeddings,values,scores
0,"[0.054808903485536575, 0.07416380941867828, -0...",Morgan Stanley,42.217224
1,"[0.17106111347675323, -0.053230009973049164, -...",Jpmorgan Chase & Co,44.135571
2,"[-0.1403375118970871, -0.25771039724349976, -0...",Mastercard Inc,47.031792
3,"[0.15066705644130707, -0.26614394783973694, -0...","Salesforce, Inc.",47.926243
4,"[-0.007676966488361359, -0.17204351723194122, ...",Wells Fargo & Company/Mn,48.350143


In [ ]:
#working HNSW
#test it
#parameter optimization
#short to short, company match
#arbitrary embeddings, BERT or mistral, openai api, SBERT (sentence bert), train your embeddings

In [13]:
len("cbfijibaaa")

10